In [ ]:
from simpy import *
import numpy as np

# Confidence Interval calculation
from scipy.stats import t
def confidence_interval(simulation_results, confidence=0.95, output_results=True):
    mean = np.mean(simulation_results)
    std = np.std(simulation_results)
    degree = len(simulation_results) - 1 
    t_crit = np.abs(t.ppf((1-confidence)/2, degree))    # Find the inverse cumulative distribution.
    ci = std*t_crit/np.sqrt(len(simulation_results))
    if output_results:
        print("Average waiting time is: ", mean)
        conf_int_low = mean-ci
        conf_int_high = mean+ci
        print("With the Confidence Interval: ", conf_int_low, conf_int_high)
        print("Confidence Interval Size: ", conf_int_high-conf_int_low)
        return
    else: return mean, ci

jobs_to_process = 300   # Max number of customers
runtime_limit = 400.0   # Rumtime limit
seed = 42               # Seed for simulation

# miu
job_rate = 10.0   # Mean time in bank
# lambda
arrival_rate = 5.0  # Mean of arrival process
# n
servers_amount = 4



def Customer(env, name, resources):
    arrive = env.now

    # Getting Status of Queues
    queue_size = []
    for queue in resources:
        queue_size.append(len(queue.put_queue) + len(queue.users))
    #print("Current Queues: ", queue_size)

    # Choosing a Queue for an arrived Client
    choice = np.argmin(queue_size)
    
    with resources[choice].request() as req:
        # Wait for the Server
        yield req
        waiting_times.append(env.now - arrive)
        # Starting the Job
        tib = np.random.exponential(job_rate)
        yield env.timeout(tib)

def Source(env, number, interval, resources):
    for i in range(number):
        c = Customer(env, 'Customer%02d' % i, resources)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)


# Starting Simulation
trials = 100
np.random.seed(seed)
waiting_times_trials = []
for i in range(trials):
    waiting_times = []      # Waiting times of performed jobs
    # Setup trial
    env = Environment()
    # Creating Queues
    resources = []
    for n in range (servers_amount):
        resources.append(Resource(env))
    env.process(Source(env, jobs_to_process, arrival_rate, resources))
    env.run(until=runtime_limit)
    
    waiting_times_trials.append(np.mean(waiting_times))     # Saving Results

confidence_interval(waiting_times_trials)